In [ ]:
import sirf.STIR as pet
import sirf
sirf.STIR.set_verbosity(1)
from sirf.Utilities import examples_data_path
import os
import sys
from cil.optimisation.functions import KullbackLeibler, OperatorCompositionFunction, IndicatorBox
cil_path = '/home/jovyan/Hackathon-000-Stochastic-Algorithms/cil/'
sys.path.append(cil_path)
from NewFISTA import ISTA

data_path = os.path.join(examples_data_path('PET'), 'thorax_single_slice')

image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
attn_image = pet.ImageData(os.path.join(data_path, 'attenuation.hv'))


acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
acq_model.set_up(template, image)
acquired_data = acq_model.forward(image)
background_term = acquired_data.get_uniform_copy(acquired_data.max()/10)
data_fit = KullbackLeibler(b=acquired_data, eta = background_term)
initial = image.get_uniform_copy(0)


f = OperatorCompositionFunction(data_fit,acq_model)

step_size = 0.1
ista = ISTA(initial=initial, f=f, g=IndicatorBox(lower=0),
                            step_size=step_size, update_objective_interval=1,
                            max_iteration=1e10, check_convergence_criterion=False)
ista.run(100)

In [ ]:
from cil.utilities.display import show2D
show2D(ista.solution)


In [ ]:
from cil.optimisation.functions import BlockFunction
from cil.optimisation.operators import BlockOperator

num_subsets = 10
acq_models = [pet.AcquisitionModelUsingRayTracingMatrix() for k in range(num_subsets)]
# create masks
im_one = image.clone()
im_one.fill(1.)
masks = []

# Loop over physical subsets
for k in range(num_subsets):
    # Set up
    acq_models[k].set_up(acquired_data, image)    
    acq_models[k].num_subsets = num_subsets
    acq_models[k].subset_num = k 

    # compute masks 
    mask = acq_models[k].direct(im_one)
    masks.append(mask)
    
data_fits = [KullbackLeibler(b=acquired_data, eta=background_term, mask=mask.as_array(), use_numba=True) for mask in masks]

F = BlockFunction(*data_fits)
K = BlockOperator(*acq_models)

f_saga = OperatorCompositionFunction(F,K)

step_size = 0.1
saga = ISTA(initial=initial, f=f_saga, g=IndicatorBox(lower=0),
                            step_size=step_size, update_objective_interval=num_subsets,
                            max_iteration=1e10, check_convergence_criterion=False)
saga.run(100*num_subsets)
